In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 93)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA50rate>80'] = (allstock.loc[id, 'RS EMA50rate'] > 80)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['RS EMA250rate>80'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['RS EMA250rate>80']].index))
    print(str(day))
    # apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:63]).prod()
    season2_profit = abs(roc_df.iloc[63:126]).prod()
    season3_profit = abs(roc_df.iloc[126:189]).prod()
    season4_profit = abs(roc_df.iloc[189:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['RS EMA250rate>80']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [7]:
avg_profit_df = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df.loc['total'] = avg_profit_df.sum()
display(avg_profit_df)

,season1_profit,season2_profit,season3_profit,season4_profit,all_profit,number of stock
2022-07-18 00:00:00,-3.708955,9.150445,22.602532,9.419916,43.063391,100.0
2022-04-15 00:00:00,-14.035776,-4.314316,9.376487,16.027386,5.039385,100.0
2022-01-12 00:00:00,-9.487399,-4.851498,-13.835899,12.651856,-17.068656,100.0
2021-10-12 00:00:00,12.081357,-1.068393,-12.269472,-8.897450,-11.362903,100.0
2021-07-12 00:00:00,-14.682251,14.593993,-4.439758,-19.328067,-25.399704,100.0
2021-04-08 00:00:00,9.096346,-9.566187,16.739575,-6.628204,1.557866,100.0
2021-01-05 00:00:00,27.621450,9.574050,-12.092559,18.028838,41.884993,100.0
2020-10-05 00:00:00,13.171917,21.876096,4.377360,-11.750882,29.264433,100.0
2020-07-03 00:00:00,2.045046,8.836122,16.472200,1.027892,32.000408,100.0
2020-04-01 00:00:00,43.040037,4.354485,9.718483,12.162307,86.475955,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 93)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA50rate>90'] = (allstock.loc[id, 'RS EMA50rate'] > 90)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS EMA250rate>70', 'RS EMA50rate>90', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:63]).prod()
    season2_profit = abs(roc_df.iloc[63:126]).prod()
    season3_profit = abs(roc_df.iloc[126:189]).prod()
    season4_profit = abs(roc_df.iloc[189:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [9]:
avg_profit_df2 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df2.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df2.loc['total'] = avg_profit_df2.sum()
display(avg_profit_df2)

,season1_profit,season2_profit,season3_profit,season4_profit,all_profit,number of stock
2022-07-18 00:00:00,-4.815885,8.216509,20.106230,13.887471,46.198307,100.0
2022-04-15 00:00:00,-10.587892,-2.348215,8.543280,16.551467,11.865500,100.0
2022-01-12 00:00:00,-9.027794,-0.940677,-9.986164,7.838750,-13.582597,100.0
2021-10-12 00:00:00,6.118097,-2.801704,-5.875702,-7.423744,-10.941729,100.0
2021-07-12 00:00:00,-15.598734,15.649064,-4.044109,-19.581964,-25.256951,100.0
2021-04-08 00:00:00,5.343734,-7.953383,18.726902,-7.301170,1.939855,100.0
2021-01-05 00:00:00,27.348516,14.380614,-13.948731,18.101851,41.923497,100.0
2020-10-05 00:00:00,14.628775,16.303130,3.152000,-9.718407,27.066449,100.0
2020-07-03 00:00:00,0.745821,9.564917,16.303736,-0.819822,31.116579,100.0
2020-04-01 00:00:00,32.229546,1.364135,5.994459,12.752593,70.208507,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-1962, -365, 93)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS MA250rate<75'] = (allstock.loc[id, 'RS 250rate']<75)
            allstock.loc[id, 'RS MA250rate>55'] = (allstock.loc[id, 'RS 250rate']>55)
            allstock.loc[id, 'RS MA20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>500k'] = (allstock.loc[id, 'Volume 50MA'] > 500*1000)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS MA20rate>85', 'RS MA250rate<75', 'RS MA250rate>55', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:63]).prod()
    season2_profit = abs(roc_df.iloc[63:126]).prod()
    season3_profit = abs(roc_df.iloc[126:189]).prod()
    season4_profit = abs(roc_df.iloc[189:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [16]:
avg_profit_df3 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df3.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df3.loc['total'] = avg_profit_df3.sum()
display(avg_profit_df3)

,season1_profit,season2_profit,season3_profit,season4_profit,all_profit,number of stock
2022-07-18 00:00:00,-3.876420,4.893419,6.351468,-1.260257,5.736450,100.0
2022-04-18 00:00:00,-1.982192,-3.150455,5.203982,10.751590,11.358124,100.0
2022-01-13 00:00:00,-5.902010,-6.389453,-5.319818,11.253888,-7.562948,100.0
2021-10-12 00:00:00,2.082938,0.608213,0.027720,-2.719573,-0.985526,100.0
2021-07-12 00:00:00,-5.625340,9.708045,0.610778,-9.883992,-8.554510,100.0
2021-04-09 00:00:00,4.581502,-2.623248,19.393335,-3.316184,14.530801,100.0
2021-01-06 00:00:00,28.579828,13.619765,-8.952025,8.980267,37.907722,100.0
2020-10-05 00:00:00,15.645263,15.615594,8.431115,-7.560091,38.851623,100.0
2020-07-06 00:00:00,-1.841862,11.005263,15.122892,-1.770629,24.141923,100.0
2020-04-06 00:00:00,4.758651,0.361941,3.931310,0.758625,13.602031,100.0


In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5000, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if i == 0:
                allstock = stockdata
                # print(allstock)
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS MA250rate<75'] = (allstock.loc[id, 'RS 250rate']<75)
            allstock.loc[id, 'RS MA250rate>55'] = (allstock.loc[id, 'RS 250rate']>55)
            allstock.loc[id, 'RS MA20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>500k'] = (allstock.loc[id, 'Volume 50MA'] > 500*1000)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            allstock.loc[id, 'new_template'] = all(allstock.loc[id, ['year low sort', 'year high sort', 'RS MA20rate>85', 'RS MA250rate<75', 'RS MA250rate>55', 'Volume 50MA>100k']])
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    apexstock = allstock.loc[allstock['new_template'], ['產業別', 'RS 250rate', 'RS 50rate', 'RS 20rate', 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA',
                                                        'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(len(allstock.loc[allstock['new_template']].index))
    print(str(day))
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/' + str(day).split(' ')[0] + 'RS選股(EMA)' + '.xlsx', encoding='utf-8-sig')


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
        if i == 0:
            rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
            # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
        else:
            rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
            rocp_df = pd.concat([rocp, rocp_df], axis=1)

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(allstock.loc[allstock['new_template']].index)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [19]:
avg_profit_df4 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df4.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df4.loc['total'] = avg_profit_df4.sum()
display(avg_profit_df4)

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2023-06-26 00:00:00,3.115078,0.000000,0.000000,0.000000,0.000000,0.000000,3.115078,100.0
2023-04-24 00:00:00,12.446426,-0.613883,0.000000,0.000000,0.000000,0.000000,11.538424,100.0
2023-02-20 00:00:00,4.165959,5.393566,1.104768,0.000000,1.104768,0.000000,12.076367,100.0
2022-12-20 00:00:00,8.251711,7.716390,0.073451,-2.839676,0.073451,-2.839676,11.517281,100.0
2022-10-19 00:00:00,0.179715,4.568159,0.464913,3.888638,0.464913,3.888638,8.602352,100.0
...,...,...,...,...,...,...,...,...
2010-05-31 00:00:00,8.331928,3.594102,-4.680124,1.537337,-4.680124,1.537337,-1.709429,100.0
2010-03-29 00:00:00,-5.903173,7.781180,3.957559,-5.769210,3.957559,-5.769210,-1.403609,100.0
2010-01-25 00:00:00,1.619578,-0.862875,1.468014,-2.992696,1.468014,-2.992696,3.311775,100.0
2009-11-24 00:00:00,6.974640,1.143012,-4.961506,6.648320,-4.961506,6.648320,1.771078,100.0


In [ ]:
avg_profit_df4.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\year low sort, year high sort, RS MA20rate大於85, RS MA250rate小於75, RS MA250rate大於55, Volume 50MA大於100k.xlsx')

In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>85)
            allstock.loc[id, 'RS EMA20rate>85'] = (allstock.loc[id, 'RS EMA20rate'] > 85)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS EMA250rate>85','RS EMA20rate>85', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.03', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    template2_df.dropna(inplace=True)
    template1_df = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-80:]
    template2_num = int(len(template1_df.index.values)/4)
    template2_df = template2_df.sort_values(by='RS EMA50rate', ascending=True).iloc[-template2_num:]
    print(f'T2 len:{len(template2_df.index.values)}')
    template2_df = template2_df.loc[~template2_df.index.isin(template1_df.index)]
    apexstock = pd.concat([template1_df, template2_df])
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'T1 len:{len(template1_df.index.values)}')
    print(f'T2 len:{len(template2_df.index.values)}')
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    


    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+365))) , '%Y-%m-%d' )
    f2 = 1
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                apexstock.drop(ID, inplace=True)
                continue
        try:
            if f2 == 1:
                rocp_df = stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP']
                # rocp_df = rocp_df.loc[(rocp_df.index<day2)]
                f2 = 0
            else:
                rocp = pd.Series(stock.loc[(stock.index>day) & (stock.index<day2), 'ROCP'], name = str(ID))
                rocp_df = pd.concat([rocp, rocp_df], axis=1)
        except Exception as e:
            print(e)
            apexstock.drop(ID, inplace=True)
            continue

    roc_df = rocp_df/100
    roc_df = roc_df+1   
    season1_profit = abs(roc_df.iloc[:42]).prod()
    season2_profit = abs(roc_df.iloc[42:84]).prod()
    season3_profit = abs(roc_df.iloc[84:126]).prod()
    season4_profit = abs(roc_df.iloc[126:168]).prod()
    season5_profit = abs(roc_df.iloc[168:210]).prod()
    season6_profit = abs(roc_df.iloc[210:]).prod()
    all_profit = abs(roc_df.iloc[:]).prod()
    season1_profit = season1_profit-1
    season2_profit = season2_profit-1
    season3_profit = season3_profit-1
    season4_profit = season4_profit-1
    season5_profit = season5_profit-1
    season6_profit = season6_profit-1
    all_profit = all_profit-1
    print(len(apexstock.index.values))
    display(season1_profit.transpose())
    print(len(season1_profit.transpose()))
    apexstock.loc[apexstock.index.values, 'profit'] = season1_profit.transpose().values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose/' + str(day).split(' ')[0] + 'RS選股(ETF)' + '.xlsx', encoding='utf-8-sig')
    season1_profit = season1_profit.sum()
    season2_profit = season2_profit.sum()
    season3_profit = season3_profit.sum()
    season4_profit = season4_profit.sum()
    season5_profit = season5_profit.sum()
    season6_profit = season6_profit.sum()
    all_profit = all_profit.sum()
    profit = {'season1_profit':season1_profit,
            'season2_profit':season2_profit,
            'season3_profit':season3_profit,
            'season4_profit':season4_profit,
            'season5_profit':season3_profit,
            'season6_profit':season4_profit,
            'all_profit':all_profit,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

In [4]:
avg_profit_df5 = pd.DataFrame([], columns=all_df.columns)
for ind in all_df.index.values:
    avg_profit_df5.loc[ind] = 100*all_df.loc[ind]/all_df.loc[ind, 'number of stock']
avg_profit_df5.loc['total'] = avg_profit_df5.sum()
display(avg_profit_df5)
avg_profit_df5.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T6.xlsx')

,season1_profit,season2_profit,season3_profit,season4_profit,season5_profit,season6_profit,all_profit,number of stock
2023-06-26 00:00:00,-0.181915,0.000000,0.000000,0.000000,0.000000,0.000000,-0.181915,100.0
2023-04-24 00:00:00,0.174609,-0.338849,0.000000,0.000000,0.000000,0.000000,-0.199739,100.0
2023-02-20 00:00:00,0.071985,-0.050683,-0.165668,0.000000,-0.165668,0.000000,-0.148455,100.0
2022-12-19 00:00:00,0.650155,-0.135312,0.295611,0.034547,0.295611,0.034547,0.891574,100.0
2022-10-18 00:00:00,-0.051190,0.160351,-0.005979,2.127206,-0.005979,2.127206,2.336502,100.0
...,...,...,...,...,...,...,...,...
2010-05-28 00:00:00,0.054328,-0.206924,-0.327700,0.226069,-0.327700,0.226069,-0.537716,100.0
2010-03-29 00:00:00,0.024055,0.100050,-0.200283,-0.336174,-0.200283,-0.336174,-0.379539,100.0
2010-01-25 00:00:00,0.137129,0.220353,0.073290,-0.171938,0.073290,-0.171938,0.204875,100.0
2009-11-23 00:00:00,1.424444,-0.114111,-0.447083,0.211215,-0.447083,0.211215,-0.440563,100.0


In [10]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA20rate>85'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA250rate<15'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA20rate<15'] = (allstock.loc[id, 'RS EMA20rate'] < 15)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.15'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS EMA250rate>85','RS EMA20rate>85', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.15', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-50:]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    TAI = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/^TWII.csv', parse_dates=['Date'], index_col=['Date'])
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            TAI_initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            TAI_final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            TAI_profit = (TAI_final_price-TAI_initial_price)/TAI_initial_price
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T6/' + str(day).split(' ')[0] + 'RS選股(T6)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1 profit':profit_df.sum().values,
              'TAIEX profit':TAI_profit,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-23 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2009-11-23 00:00:00


Date
2009-11-24    19.485565
2009-11-25    20.827906
Name: Adj Close, dtype: float64

Date
2010-01-20    23.685791
2010-01-21    22.213547
Name: Adj Close, dtype: float64

Date
2009-11-24    19.485565
2009-11-25    20.827906
Name: Adj Close, dtype: float64

Date
2010-01-20    23.685791
2010-01-21    22.213547
Name: Adj Close, dtype: float64

50
(0,)


ValueError: shape mismatch: value array of shape (0,) could not be broadcast to indexing result of shape (50,)

In [9]:
display(all_df)
avg_profit_df5 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df5.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1 profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df5.loc[ind, 'TAIEX profit'] = all_df.loc[ind, 'TAIEX profit']*100
    avg_profit_df5.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df5.loc['total'] = avg_profit_df5.sum()
display(avg_profit_df5)
avg_profit_df5.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T6.xlsx')

,season1 profit,TAIEX profit,number of stock
2023-06-26,-1.045387,-0.115621,50
2023-04-24,7.145901,0.220867,50
2023-02-20,4.412379,-0.007143,50
2022-12-19,5.675999,0.110860,50
2022-10-18,3.673772,-0.028846,50
...,...,...,...
2010-07-29,1.507024,-0.188608,50
2010-05-28,2.392845,-0.221913,50
2010-03-29,-3.695570,0.179032,50
2010-01-25,6.420331,-0.123932,50


,season1 profit,TAIEX profit,numbers of stock
2023-06-26 00:00:00,-2.090775,-11.562116,50.0
2023-04-24 00:00:00,14.291803,22.086712,50.0
2023-02-20 00:00:00,8.824758,-0.714289,50.0
2022-12-19 00:00:00,11.351998,11.085968,50.0
2022-10-18 00:00:00,7.347545,-2.884622,50.0
...,...,...,...
2010-05-28 00:00:00,4.785689,-22.191334,50.0
2010-03-29 00:00:00,-7.391140,17.903211,50.0
2010-01-25 00:00:00,12.840662,-12.393160,50.0
2009-11-23 00:00:00,18.937851,63.265326,50.0


In [14]:
TAI = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/^TWII.csv', parse_dates=['Date'], index_col=['Date'])
TAI_profit = []
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
    final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
    TAI_initial_price = TAI.loc[TAI.index>day, 'Adj Close'].iloc[0]
    TAI_final_price = TAI.loc[TAI.index<day2, 'Adj Close'].iloc[-1]
    TAI_profit.append((TAI_final_price-TAI_initial_price)/TAI_initial_price)
    print(day,day2)
    print(TAI_initial_price,TAI_final_price)
print(TAI_profit)

2009-11-23 00:00:00 2010-01-24 00:00:00
7714.53125 7927.279785
2010-01-24 00:00:00 2010-03-27 00:00:00
7872.960449 7876.830078
2010-03-27 00:00:00 2010-05-28 00:00:00
7947.419922 7243.132813
2010-05-28 00:00:00 2010-07-29 00:00:00
7373.952148 7784.78125
2010-07-29 00:00:00 2010-09-29 00:00:00
7760.601074 8189.40918
2010-09-29 00:00:00 2010-11-30 00:00:00
8237.748047 8367.137695
2010-11-30 00:00:00 2011-01-31 00:00:00
8520.078125 9145.31543
2011-01-31 00:00:00 2011-04-03 00:00:00
9111.425781 8705.097656
2011-04-03 00:00:00 2011-06-04 00:00:00
8851.948242 9046.24707
2011-06-04 00:00:00 2011-08-05 00:00:00
9057.06543 8317.237305
2011-08-05 00:00:00 2011-10-06 00:00:00
7552.770996 6989.124023
2011-10-06 00:00:00 2011-12-07 00:00:00
7211.932617 6956.253906
2011-12-07 00:00:00 2012-02-07 00:00:00
6982.874023 7687.951172
2012-02-07 00:00:00 2012-04-09 00:00:00
7869.880371 7706.230957
2012-04-09 00:00:00 2012-06-10 00:00:00
7640.651367 6999.624023
2012-06-10 00:00:00 2012-08-11 00:00:00
7120.2

In [21]:
T9excel = pd.read_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T9.xlsx')
T9excel.set_index('Unnamed: 0', inplace=True)
# TAI_profit.append(np.sum(TAI_profit))
# T9excel['TAIEX'] = TAI_profit
T9excel['TAIEX'] = T9excel['TAIEX']*100
print(T9excel)
T9excel.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T9.xlsx')

                      T9 profit   T1 profit  numbers of stock     T1-T9  \
Unnamed: 0                                                                
2009-11-23 00:00:00   14.677741   18.937851                50  4.260110   
2010-01-25 00:00:00   10.161955   12.840662                50  2.678707   
2010-03-29 00:00:00   -6.511397   -7.391140                50 -0.879743   
2010-05-28 00:00:00    7.750917    4.785689                50 -2.965227   
2010-07-29 00:00:00    0.918534    3.014049                50  2.095515   
...                         ...         ...               ...       ...   
2022-12-19 00:00:00   12.923249   11.351998                50 -1.571251   
2023-02-20 00:00:00    8.824758    8.824758                50  0.000000   
2023-04-24 00:00:00   14.807163   14.291803                50 -0.515360   
2023-06-26 00:00:00   -3.400260   -2.090775                50  1.309485   
total                196.150579  235.668417              4011       NaN   

                        

In [22]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 62)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            allstock.loc[id, 'RS 50rate>80'] = (allstock.loc[id, 'RS 50rate'] > 80)
            allstock.loc[id, 'RS EMA250rate>85'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA20rate>85'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'RS EMA250rate<15'] = (allstock.loc[id, 'RS EMA250rate']>80)
            allstock.loc[id, 'RS EMA20rate<15'] = (allstock.loc[id, 'RS EMA20rate'] < 15)
            allstock.loc[id, 'RS EMA250rate>70'] = (allstock.loc[id, 'RS EMA250rate']>70)
            allstock.loc[id, 'RS EMA20rate>75'] = (allstock.loc[id, 'RS EMA20rate'] > 75)
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.15'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            allstock.loc[id, 'RS 20rate>70'] = (allstock.loc[id, 'RS 20rate'] > 70)
            allstock.loc[id, 'RS 250rate>70'] = (allstock.loc[id, 'RS 250rate'] > 70)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            allstock.loc[id, 'Volume 50MA>1000k'] = (allstock.loc[id, 'Volume 50MA'] > 1000*1000)
            allstock.loc[id, 'Mark Tamplate'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
            allstock.loc[id, 'T1'] = all(allstock.loc[id, ['RS EMA250rate>85','RS EMA20rate>85', 'RS EMA20rate<99', 'Volume 50MA>100k']])
            allstock.loc[id, 'T2'] = all(allstock.loc[id, ['ATR250/price<0.15', 'Volume 50MA>1000k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 第一個日期選出符合條件的股票並匯出excel
    # print(allstock['Mark Tamplate'])
    allstock.dropna(subset='Mark Tamplate', inplace=True)
    template1_df = allstock.loc[allstock['T1'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template2_df = allstock.loc[allstock['T2'], ['產業別', 'T1', 'T2', 'ATR250/price', 'ATR50/price', 'ATR20/price'
                                                 , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                                 , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA'
                                                 ,'year high sort', 'year low sort', '200MA trending up 20d', '200MA trending up 60d', 'RS 250rate>70', 'RS 250rate>80', 'Volume 50MA>100k']]
    template1_df.dropna(inplace=True)
    apexstock = template1_df.sort_values(by='RS EMA250rate', ascending=True).iloc[-50:]
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    apexstock = apexstock.dropna()
    print(f'apexstock len:{len(apexstock.index.values)}')
    print(str(day))
    

    profit_list = []
    IDs = apexstock.index.values
    day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' )
    TAI = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/^TWII.csv', parse_dates=['Date'], index_col=['Date'])
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        try:
            stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
        except:
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
            except Exception as e:
                print(ID)
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                except:
                    pass
                continue
        try:
            initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            TAI_initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
            TAI_final_price = stock.loc[stock.index<day2, 'Adj Close'].iloc[-1]
            TAI_profit = (TAI_final_price-TAI_initial_price)/TAI_initial_price
            profit_list.append([ID, (final_price-initial_price)/initial_price])
        except Exception as e:
            print(e)
            try:
                apexstock.drop(ID, inplace=True)
            except:
                pass
            continue

    print(len(apexstock.index.values))
    print(np.shape(profit_list))
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, 'profit'] = profit_df['profit'].values
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose T6-/' + str(day).split(' ')[0] + 'RS選股(T6-)' + '.xlsx', encoding='utf-8-sig')
    profit = {'season1 profit':profit_df.sum().values,
              'TAIEX profit':TAI_profit,
            'number of stock':len(apexstock.index.values)}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)
    # print(season1_profit.sum())
    # print(season2_profit.sum())
    # print(season3_profit.sum())
    # print(season4_profit.sum())
    # print(all_profit.sum())

     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]
day 2009-11-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2009-11-24 00:00:00
50
(50, 2)


,season1 profit,TAIEX profit,number of stock
2009-11-24,8.210346,0.098039,50


day 2010-01-25 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-01-25 00:00:00
50
(50, 2)


,season1 profit,TAIEX profit,number of stock
2010-01-25,6.050739,-0.123932,50
2009-11-24,8.210346,0.098039,50


day 2010-03-29 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2010-03-29 00:00:00
50
(50, 2)


...

,season1 profit,TAIEX profit,number of stock
2023-02-20,5.709771,-0.007143,50
2022-12-20,7.307742,0.100897,50
2022-10-19,4.641322,-0.067857,50
2022-08-18,-7.751963,-0.113150,50
2022-06-17,7.407422,0.045278,50
...,...,...,...
2010-07-30,1.119467,-0.137097,50
2010-05-31,2.348902,-0.075130,50
2010-03-29,-3.644393,0.155556,50
2010-01-25,6.050739,-0.123932,50


day 2023-04-24 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-04-24 00:00:00
50
(50, 2)


,season1 profit,TAIEX profit,number of stock
2023-04-24,6.156398,0.220867,50
2023-02-20,5.709771,-0.007143,50
2022-12-20,7.307742,0.100897,50
2022-10-19,4.641322,-0.067857,50
2022-08-18,-7.751963,-0.113150,50
...,...,...,...
2010-07-30,1.119467,-0.137097,50
2010-05-31,2.348902,-0.075130,50
2010-03-29,-3.644393,0.155556,50
2010-01-25,6.050739,-0.123932,50


day 2023-06-26 00:00:00
'<' not supported between instances of 'numpy.ndarray' and 'str'
6170.0 失敗
apexstock len:50
2023-06-26 00:00:00
50
(50, 2)


,season1 profit,TAIEX profit,number of stock
2023-06-26,0.881193,0.104670,50
2023-04-24,6.156398,0.220867,50
2023-02-20,5.709771,-0.007143,50
2022-12-20,7.307742,0.100897,50
2022-10-19,4.641322,-0.067857,50
...,...,...,...
2010-07-30,1.119467,-0.137097,50
2010-05-31,2.348902,-0.075130,50
2010-03-29,-3.644393,0.155556,50
2010-01-25,6.050739,-0.123932,50


In [24]:
display(all_df)
avg_profit_df6 = pd.DataFrame([])
for ind in all_df.index.values:
    avg_profit_df6.loc[ind, 'season1 profit'] = 100*all_df.loc[ind, 'season1 profit']/all_df.loc[ind, 'number of stock']
    avg_profit_df6.loc[ind, 'numbers of stock'] = all_df.loc[ind, 'number of stock']
avg_profit_df6.loc['total'] = avg_profit_df6.sum()
display(avg_profit_df6)
avg_profit_df6.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\T6-.xlsx')

,season1 profit,TAIEX profit,number of stock
2023-06-26,0.881193,0.104670,50
2023-04-24,6.156398,0.220867,50
2023-02-20,5.709771,-0.007143,50
2022-12-20,7.307742,0.100897,50
2022-10-19,4.641322,-0.067857,50
...,...,...,...
2010-07-30,1.119467,-0.137097,50
2010-05-31,2.348902,-0.075130,50
2010-03-29,-3.644393,0.155556,50
2010-01-25,6.050739,-0.123932,50


,season1 profit,numbers of stock
2023-06-26 00:00:00,1.762385,50.0
2023-04-24 00:00:00,12.312797,50.0
2023-02-20 00:00:00,11.419541,50.0
2022-12-20 00:00:00,14.615484,50.0
2022-10-19 00:00:00,9.282644,50.0
...,...,...
2010-05-31 00:00:00,4.697804,50.0
2010-03-29 00:00:00,-7.288786,50.0
2010-01-25 00:00:00,12.101478,50.0
2009-11-24 00:00:00,16.420692,50.0
